
# Setup for Pangeo Environment on Raijin


In this notebook we will go through:

- Configuring the Pangeo environment on your Raijin account
- Submitting a multi-node Pangeo job to the queue system
- Using Jupyter dashboard for loading notebooks and monitoring jobs


The [Pangeo](http://pangeo.io) software ecosystem involves open source tools such as [xarray](http://xarray.pydata.org/en/stable/), [iris](https://scitools.org.uk/iris/docs/latest/), [dask](https://dask.org/), [Jupyter](https://jupyter.org/) and many other packages. 

This notebook provides instructions on how to use the Pangeo environment to run a multi-node parallel Jupyter notebook within the queues of Raijin and shows how to interact with it from your desktop.

Please note that Pangeo will soon be enabled on Gadi.

## Configuring your account on Raijin

###  Step 1: Enabling Pangeo in your shell envorinment

To enable the Pangeo environment, you will use the following commands within jobs, or within an interactive environment:

```
$ module load pangeo/2019.10
$ source ${PANGEO_ROOT}/etc/profile.d/conda.sh
$ conda activate pangeo
```
You should now see Pangeo appear in the brackets in front of the prompt. Note that Pangeo has its own python installation.


![1](images/pangeo_setup1.png)

![2](images/pangeo_setup2.png)

###  Step 2: Configuring your JupyterLab password on Raijin

We will use JupyterLab to load notebooks and monitor jobs. JupyterLab is bundled within the Pangeo environment. To setup this environment run the following two commands:

```
(pangeo) $ jupyter notebook --generate-config
(pangeo) $ jupyter notebook password
```

This is a stand-alone password that you will use later for accessing the JupyterLab server. You can use this command to reset this password at any time.

### Step 3: Exiting the Pangeo environment

You can quit the environment at any time by using "conda deactivate":
```
$ conda deactivate
```

### Submitting a multi-node Pangeo job to Raijin

First create a directory where you will run the jupyter notebook:

```
$ mkdir ~/pangeo/tutorial

```
You can create a shell script by copying the following commands into a script file. Let’s name this file run_ipynb_job.sh:

```
#!/bin/bash
#PBS -N pangeo_test
#PBS -P c25
#PBS -q normal
#PBS -l walltime=5:00:00
#PBS -l ncpus=32
#PBS -l mem=64GB
#PBS -l jobfs=100GB
module load pangeo/2019.10
pangeo.ini.all.sh
sleep infinity
```

Dask requires whole nodes so you need to have a job that needs these resources. In this example we request 2 Raijin Sandy Bridge nodes with 32 CPU and 64GB memory. For further instructions about node types on Raijin, consult the [NCI user guide](https://opus.nci.org.au/display/Help/Running+Jobs) on the specific configurations. 


The above jobscript will load the Pangeo module, run the initialization script called **pangeo.ini.all.sh**, and keeps the environment alive for the lifetime of the PBS job. The initialization script will set up the dask scheduler on one node and multiple workers on all the other nodes. 

Now, submit the job to the queue:

```
$ qsub run_ipynb_job.sh

```

and take note of your job_id (which may look something like *3139991.r-man2*). Check the queue to see when the job is running:

```
$ qstat <job_id>

```


![3](images/pangeo_setup3.png)



### Set up port forwarding to access from your desktop machine

Once the job is running on Raijin, there are two files that will appear in your current Raijin directory: 

* client_cmd
* scheduler.json

The file client_cmd contains commands to forward network traffic from the defined port number of the worker node to an external client machine (i.e., your desktop) via the login node raijin.nci.org.au. In the example below, JupyterLab uses port 8343 and dask dashboard occupies port 8890 respectively from Raijin worker node r225. Note both port numbers are randomly chosen for each job – they will be different each time you run a job.

![4](images/pangeo_setup4.png)


Next, on your remote machine (e.g., Desktop), run each command separately and enter your Raijin password when prompted.

CLI_1: 

```
$ ssh -N -L 8343:r225:8343 jbw900@raijin.nci.org.au 
jbw900@raijin.nci.org.au's password: 
```

Then "ctrl Z" and then type 

```
$ bg
```

CLI_2: 

```
$ ssh -N -L 8890:r225:8890 jbw900@raijin.nci.org.au 
jbw900@raijin.nci.org.au's password: 
```

Then "ctrl Z" and then type 

```
$ bg
```

### Connect to the remote JupyterLab server from your remote desktop computer

In a browser on your local desktop, type the following URL “localhost:8343” – where the port matches the details above:

![6](images/pangeo_setup6.png)

You will be prompted for your JupyterLab password that you created earlier:
![7](images/pangeo_setup7.png)

Once your authentication has passed, a JupyterLab interface will be launched in a few seconds.

![8](images/pangeo_setup8.png)

### Importing a notebook example

You can drag and drop a notebook from your local computer into this JupyterLab. Then the file will also appear in your working directory in Raijin:
![9](images/pangeo_setup9.png)

The screen shot above shows

1. Jupyter notebook interface opened from your local computer and connecting to Raijin
2. A directory on local computer from where a notebook is dragged and dropped into the JupyterLab interface
3. Raijin command window showing that the notebook appears instantly

### Setting up your Jupyter notebook to use the dask server

**At the beginning of the notebook**

To utilise the dask server established from the PBS job, it is necessary to add and run the following cell at the beginning of your notebook: 

```
from dask.distributed import Client,LocalCluster 
client = Client(scheduler_file='scheduler.json') 
print(client) 
```

The output will show the configuration of the client and dask cluster. You can check that the number of cores matches what you requested in the job script. Now you could run your notebook as usual.

**At the end of the jobscript**

Add and run a cell as below to gracefully stop the job:

```
!pangeo.end.sh
```

### Recap important notes

Please make sure the following start and stop instructions are added at the beginning and the end of the notebook respectively.

```
# start the dask client
client =  Client(scheduler_file='scheduler.json')
 
# stop the pbs job.
! pangeo.end.sh
```

### View compute threads using Dask dashboard

From your local desktop, open a new tab in the web browser, type the following, the second port in the client_cmd file to open the Dask dashboard. This will allow you to see the dynamic resources of the processing.  In your Jupyter notebook, a computationally intensive task will be recognised with a (*) at the start of a cell that is currently in execution.  In the Dask dashboard you would then see something similar to the following:

```
localhost:8890
```

![10](images/pangeo_setup10.png)

### Run Jupyter notebook in a batch job

Of course you don’t need to run jupyter notebooks interactively. You can just submit as a job.

### Step 1: convert your jupyter notebook to a python script

```
$ jupyter nbconvert --to script [YOUR_NOTEBOOK}.ipynb
```

Make sure you have added the following lines at the beginning of the python script.

```
from dask.distributed import Client,LocalCluster 
client = Client(scheduler_file='scheduler.json') 
print(client) 
```


### Step 2: Create the job script as below

```
#!/bin/bash 
#PBS -N pangeo_test 
#PBS -q normal
#PBS -P c25
#PBS -l walltime=5:00:00 
#PBS -l ncpus=32 
#PBS -l mem=64GB 
#PBS -l jobfs=100GB 
 
module load pangeo/2019.10 
pangeo.ini.all.sh 
source ${PANGEO_ROOT}/etc/profile.d/conda.sh 
conda activate pangeo 
 
cd $PBS_O_WORKDIR 
python YOUR_PYSCRIPT_NAME.py 
pangeo.end.sh
```

Modify parameters that suit your case, and name it as **run_py.sh**

### Step 3: Submit your job script via 'qsub' command


```
$ qsub run_py.sh
```

### Reference

- http://pangeo.io